In [1]:
import torch

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
else:
    print("No GPUs available.")

Number of GPUs available: 1


In [2]:
%pip install imageio

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [5]:
from torchvision import datasets, transforms
import os
import gdown

In [16]:
image_size = 64
batch_size = 64
num_epochs = 50
lr = 0.0002
beta1 = 0.5
save_interval  = 5
max_checkpoints = 50  # Keep last 50 checkpoints

checkpoint_dir = "./checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)


# Create data directory
data_dir = "./data/celeba_hq"
os.makedirs(data_dir, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [18]:
! ls ./data/

celeba_hq


In [12]:


# ✅ Replace these IDs with *real* Google Drive file IDs (must be public)
file_ids = [
    "1hIjAnuef_V6iJgLlgvC7mGNvhrxoQhYY",  # Example: celeba img_align_celeba.zip
    "17Z1YCKTBIRmGLfE0qAq057fjk6VtdqJk",  # Example: list_attr_celeba.txt
    "1VM9XoqDjLOq75XFEYzGYA83ri6fhsSye"   # Example: list_eval_partition.txt
]

# ✅ Loop through and download + unzip each file
for idx, fid in enumerate(file_ids):
    if idx == 0:
        url = f"https://drive.google.com/uc?id={fid}"
        zip_path = os.path.join(data_dir, f"CelebA-HQ-{idx}.zip")
    
        print(f"Downloading chunk {idx + 1} from {url} ...")
        try:
            gdown.download(url, zip_path, quiet=False)
        except Exception as e:
            print(f"⚠️ Error downloading file {fid}: {e}")
            continue
    
        if os.path.exists(zip_path):
            print(f"Extracting chunk {idx + 1}...")
            os.system(f'unzip -q "{zip_path}" -d "{data_dir}"')
            os.remove(zip_path)
        else:
            print(f"⚠️ Skipping extraction for chunk {idx + 1}, file missing.")
    else:
        url = f"https://drive.google.com/uc?id={fid}"
        zip_path = os.path.join(data_dir, f"CelebA-HQ-{idx}.txt")
    
        print(f"Downloading chunk {idx + 1} from {url} ...")
        try:
            gdown.download(url, zip_path, quiet=False)
        except Exception as e:
            print(f"⚠️ Error downloading file {fid}: {e}")
            continue
        

print("✅ All chunks downloaded and extracted successfully!")


Downloading...
From: https://drive.google.com/uc?id=17Z1YCKTBIRmGLfE0qAq057fjk6VtdqJk
To: /data/celeba_hq/CelebA-HQ-1.txt
100%|██████████| 26.7M/26.7M [00:10<00:00, 2.62MB/s]


Downloading...
From: https://drive.google.com/uc?id=1VM9XoqDjLOq75XFEYzGYA83ri6fhsSye
To: /data/celeba_hq/CelebA-HQ-2.txt
100%|██████████| 12.2M/12.2M [00:01<00:00, 9.28MB/s]

✅ All chunks downloaded and extracted successfully!


In [14]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import gdown
import matplotlib.pyplot as plt
import imageio
from IPython.display import Image
from torchvision import utils as vutils


In [15]:
transform = transforms.Compose([
    transforms.CenterCrop(178),
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

print("Dataset size:", len(dataset))

Dataset size: 202599
